## Optimization per Stadsdeel
The aim of this Notebook is to explore the possibiliy to optimize certain subsets of the problem and to reassamble these results to see whether or not this actually helps in obtaining a valid, improved solution. 

### Current situation
The process starts with the normal loading of the data, followed by a calculation of the average distance and penalties as a way to set a baseline. 

In [1]:
import requests
import pandas as pd
import geopandas as gpd

import sys

sys.path.append("../")

from Code.helper_functions import initial_loading, analyze_candidate_solution
from Code.algorithms import random_start_hillclimber
from Code.loading_data import create_all_households

POSTGRES password?WelkomCorne!


In [2]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()

Do you want to use addresses instead of clusters?True
What stadsdeel do you want to make as a subsection(optional parameter)?
What is the maximum amount of containers in a cluster that is considered to be useful?8
Where to get db files(local/online)?local
DB relation POIs loaded


..\Code\loading_data.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3])\
..\Code\loading_data.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ligtin_bag_pnd_identificatie'] = \
..\Code\loading_data.py:369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

distance matrix loaded
API data loaded
Table all households created
API and DB joined
containers per cluster determined
move_rest determined


In [3]:
joined = joined[joined['totaal'] < 9]

In [4]:
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=False, use_count=True)

Average distance is : 146.28882503911674
Penalties are: 101.24041800456227


In [ ]:
hillclimber_df, best_solution = random_start_hillclimber(joined, all_households, rel_poi_df, df_afstandn2)

How many random iterations?200
How many iterations hillclimber?1000
Do you want the results saved(True/False)?True
Do you want to only use a subset of data?False
Do you want to use addresses over clusters?True
Do you want to apply simulated annealing? (True/False)False
Optimize on (score/penalties)?penalties
What method (2-opt, dim Gaussian, Gaussian)?Gaussian
Average distance is : 146.28882503911674
Penalties are: 101.24041800456227
Average distance is : 160.40389711435898
Penalties are: 109.4467854108987
Average distance is : 158.4027773845703
Penalties are: 108.62278521741888
Average distance is : 160.34288950083345
Penalties are: 110.26150308697069
Average distance is : 159.2481099013852
Penalties are: 110.46134858187347
Average distance is : 159.96970177678892
Penalties are: 110.06956009545472
Average distance is : 159.77538101659846
Penalties are: 108.41323753137951
Average distance is : 158.66781787360986
Penalties are: 109.46688410646294
Average distance is : 158.82582946387808

Average distance is : 160.0684251512266
Penalties are: 110.5116581215045
Average distance is : 161.2072656296176
Penalties are: 110.15947228523285
Average distance is : 159.72428090722752
Penalties are: 110.36272388914405
Average distance is : 159.76319003578777
Penalties are: 111.03128056648694
Average distance is : 160.51230116435988
Penalties are: 108.6863251583714
Average distance is : 158.53120844412513
Penalties are: 107.82918512744745
Average distance is : 159.91152476820355
Penalties are: 108.049105444797
Average distance is : 159.90384623603074
Penalties are: 108.78585057337148
Average distance is : 161.4624681729585
Penalties are: 109.6184478889655
Average distance is : 159.53875679567093
Penalties are: 109.75739062007695
Average distance is : 161.04211915051414
Penalties are: 110.2154091044985
Average distance is : 160.39031070335392
Penalties are: 109.68710185586481
Average distance is : 157.68353542519517
Penalties are: 109.74251336930017
Average distance is : 158.73548692

Average distance is : 146.9508419857144
Penalties are: 101.35663559213363
101.35663559213363 100.9219147163306
Amount of clusters to change: 182
Average distance is : 147.05763582131718
Penalties are: 101.1522350890443
101.1522350890443 100.9219147163306
Amount of clusters to change: 157
Average distance is : 146.79384724497208
Penalties are: 100.91556210190582
100.91556210190582 100.9219147163306
Amount of clusters to change: 153
Average distance is : 146.9765995415098
Penalties are: 100.91485610161064
100.91485610161064 100.91556210190582
Amount of clusters to change: 164
Average distance is : 147.22170400334267
Penalties are: 101.13503567637453
101.13503567637453 100.91485610161064
Amount of clusters to change: 157
Average distance is : 147.56919524124865
Penalties are: 101.47701602601181
101.47701602601181 100.91485610161064
Amount of clusters to change: 154
Average distance is : 146.819837973899
Penalties are: 100.90688810982297
100.90688810982297 100.91485610161064
Amount of clus

Average distance is : 147.57936081407433
Penalties are: 99.00081186279918
99.00081186279918 99.00516451696275
Amount of clusters to change: 139
Average distance is : 147.63587061780143
Penalties are: 99.07731131990396
99.07731131990396 99.00081186279918
Amount of clusters to change: 165
Average distance is : 147.829348940545
Penalties are: 99.45505746729258
99.45505746729258 99.00081186279918
Amount of clusters to change: 108
Average distance is : 147.8195357460854
Penalties are: 99.37959478530667
99.37959478530667 99.00081186279918
Amount of clusters to change: 133
Average distance is : 147.70386584421482
Penalties are: 99.11553941189162
99.11553941189162 99.00081186279918
Amount of clusters to change: 167
Average distance is : 147.4434946610789
Penalties are: 98.60614258369769
98.60614258369769 99.00081186279918
Amount of clusters to change: 159
Average distance is : 148.14493043652524
Penalties are: 99.46258941087052
99.46258941087052 98.60614258369769
Amount of clusters to change: 

### Modification
This step consists of remvoving all information regarding the configuration of the containers in a certain stadsdeel, optimizing a copy of this and resetting it back into the initial solution to see its impact. The first optimization is done in stadsdeel Zuidoost (T), as it contains the least amount of garbage clusters

In [ ]:
joined_T = joined[joined['stadsdeel'] == 'T']

In [ ]:
all_households_T  = create_all_households(rel_poi_df, subsectie='T')
all_households_T = all_households_T.rename(columns={'s1_afv_nodes': 'naar_s1_afv_nodes'})

In [ ]:
hillclimber_df_T, best_solution_T = random_start_hillclimber(joined_T, all_households_T, rel_poi_df, df_afstandn2)

In [ ]:
# Join improved solution back with the rest and calculate score
joined_new = joined[joined['stadsdeel'] != 'T']
joined_new = joined_new.append(best_solution, ignore_index=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined_new, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)

### Conclusion
This system of isolating certain stadsdelen appears to be working just fine and can be applied further to be used for multiple (clusters of) stadsdelen. The obvious places for optimization include Zuidoost (T), Noord (N), Nieuw-West(F) and the remaining cluster of Oost (M), Zuidoost(T), West(E) and Centrum(A). This is applied next

### Optimization of Zuid-Oost (T)

In [ ]:
joined_T = joined[joined['stadsdeel'] == 'T']
all_households_T  = create_all_households(rel_poi_df, subsectie='T')
all_households_T = all_households_T.rename(columns={'s1_afv_nodes': 'naar_s1_afv_nodes'})

In [ ]:
hillclimber_df_T, best_solution_T = random_start_hillclimber(joined_T, all_households_T, rel_poi_df, df_afstandn2)

In [ ]:
plt = hillclimber_df_T['best'].plot(title='hillclimber of Zuid-Oost')
plt.set_xlabel('Number of iterations')
plt.set_ylabel('Penalty score')
plt.figure.savefig('20200505_ZuidOost.pdf')

In [ ]:
# Join improved solution back with the rest and calculate score
joined = joined[joined['stadsdeel'] != 'T']
joined = joined.append(best_solution_T, ignore_index=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)

### Optimization of Noord (N)

In [ ]:
joined_N = joined[joined['stadsdeel'] == 'N']
all_households_N  = create_all_households(rel_poi_df, subsectie='N')
all_households_N = all_households_N.rename(columns={'s1_afv_nodes': 'naar_s1_afv_nodes'})

In [ ]:
hillclimber_df_N, best_solution_N = random_start_hillclimber(joined_N, all_households_N, rel_poi_df, df_afstandn2)

In [ ]:
plt = hillclimber_df_N['best'].plot(title='hillclimber of Noord')
plt.set_xlabel('Number of iterations')
plt.set_ylabel('Penalty score')
plt.figure.savefig('20200505_Noord.pdf')

In [ ]:
# Join improved solution back with the rest and calculate score
joined = joined[joined['stadsdeel'] != 'N']
joined = joined.append(best_solution_N, ignore_index=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)

### Optimization of Nieuw-West (F)

In [ ]:
joined_F = joined[joined['stadsdeel'] == 'F']
all_households_F  = create_all_households(rel_poi_df, subsectie='F')
all_households_F = all_households_F.rename(columns={'s1_afv_nodes': 'naar_s1_afv_nodes'})

In [ ]:
hillclimber_df_F, best_solution_F = random_start_hillclimber(joined_F, all_households_F, rel_poi_df, df_afstandn2)

In [ ]:
plt = hillclimber_df_F['best'].plot(title='hillclimber of Nieuw-West')
plt.set_xlabel('Number of iterations')
plt.set_ylabel('Penalty score')
plt.figure.savefig('20200505_NieuwWest.pdf')

In [ ]:
# Join improved solution back with the rest and calculate score
joined = joined[joined['stadsdeel'] != 'F']
joined = joined.append(best_solution_F, ignore_index=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)

### Optimization of "Centrum"

In [ ]:
joined_C = joined[joined['stadsdeel'].isin(['M', 'A', 'K', 'E'])]
all_households_C = create_all_households(rel_poi_df, subsectie = ['M', 'A', 'K', 'E'])
all_households_C = all_households_C.rename(columns={'s1_afv_nodes': 'naar_s1_afv_nodes'})

In [ ]:
hillclimber_df_C, best_solution_C = random_start_hillclimber(joined_C, all_households_C, rel_poi_df, df_afstandn2)

In [ ]:
plt = hillclimber_df_C['best'].plot(title='hillclimber of "Centrum"')
plt.set_xlabel('Number of iterations')
plt.set_ylabel('Penalty score')
plt.figure.savefig('20200507_Centrum.pdf')

In [ ]:
# Join improved solution back with the rest and calculate score
joined = joined[joined['stadsdeel'].isin(['T', 'N', 'F'])]
joined = joined.append(best_solution_C, ignore_index=True)
joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)

In [ ]:
lst = [295.12, 286.96, 271.81, 258.44, 232.13]
pd.DataFrame(lst).plot()

### Conclusion
Preliminary results show that optimization for the different stadsdelen proves to be efficient and working. A new step is to compare the initial results of the current situation with the final score and to see what makes the difference. Another thing to investigate is the huge performance optimization of the Zuidoost part

In [ ]:
# joined_total = joined_start.set_index('s1_afv_nodes').join(joined.set_index('s1_afv_nodes'), rsuffix='_new', lsuffix='_old')
# joined_total['rest_difference'] = -(joined_total['rest_new'] - joined_total['rest_old'])
# joined_total['plastic_difference'] = -(joined_total['plastic_new'] - joined_total['plastic_old'])
# joined_total['paper_difference'] = -(joined_total['papier_new'] - joined_total['papier_old'])
# joined_total['glass_difference'] = -(joined_total['glas_new'] - joined_total['glas_old'])
# joined_total['textile_difference'] = -(joined_total['textiel_new'] - joined_total['textiel_old'])
# # joined_total[['rest_difference', 'plastic_difference', 'paper_difference', 'glass_difference', 'textile_difference', 'totaal_new']]
# tmp = joined_total[(joined_total['totaal_old'] > 10)][['cluster_x_old', 'cluster_y_old', 'rest_old', 'rest_new', 'totaal_old', 'totaal_new', 'stadsdeel_old']]
# print(tmp['stadsdeel_old'].value_counts())
# joined_total.sum()

### Calculation of lower and upper bounds

In [ ]:
# Lower bound
joined_cluster_distance = joined.set_index('s1_afv_nodes').join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index().rename(columns={'index': 'van_s1_afv_nodes'})
joined_cluster_distance = joined_cluster_distance.sort_values(by='afstand')
pivot = joined_cluster_distance.groupby('naar_s1_afv_nodes').first()
lower_bound = (pivot['afstand'] * pivot['count']).sum() / pivot['count'].sum()
lower_bound